In [1]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
import requests
from io import BytesIO

# Load environment variables from the .env file
load_dotenv()

# Retrieve the API key from the environment variable
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
generation_config = genai.GenerationConfig(
    temperature=0.0,
    response_mime_type="application/json"
)

In [2]:
system_instruction_prompt= """
You are a Cognitive Specialist AI. Your task is to refer to the attached papers, then generate factual, score‑driven descriptions for a “Child Profile” JSON object. Do not invent data or hallucinate—only use established research-based language.  

When prompted, you will receive:  
1. One or more uploaded PDF resource papers.  
2. A JSON skeleton with numeric scores and empty description fields (as in the example below).  

Your job is to populate every `"description"` or summary and also modalities of the kids profile field based strictly on the score values and established cognitive‑domain definitions in the reference material. Output **only** the completed JSON object—no extra commentary, no markdown, and no omitted fields.  

Example input schema:
```json
{
  "name": "Kid",
  "gender": "",
  "age": 1,
  "diagnosis": "not identified",
  "cognitiveDomainProfile": {
    "workingMemory": {
      "score": 72,
      "description": ""
    },
    "processingSpeed": {
      "score": 68,
      "description": ""
    },
    "attentionSpan": {
      "visualAttention": {
        "score": 65,
        "description": ""
      },
      "auditoryAttention": {
        "score": 70,
        "description": ""
      }
    }
  },
  "executiveFunction": {
    "planning": 4,
    "impulseControl": 3,
    "description": ""
  },
  "learningPreferences": {
    "modalities": [],
    "description": ""
  },
  "learningChallengesSummary": {
    "attentionAndFocus": "",
    "memoryAndSpeed": "",
    "planningAndImpulsivity": "",
    "strengths": ""
  }
}


 """

In [7]:
profile1 = {
  "name": "Kid",
  "gender": "male",
  "age": 13,
  "diagnosis": "Not formally diagnosed, but exhibiting early symptoms suggestive of attention deficit hyperactivity disorder (ADHD)",
  "cognitiveDomainProfile": {
    "workingMemory": {
      "score": 85,
      "description": ""
    },
    "processingSpeed": {
      "score": 78,
      "description": ""
    },
    "attentionSpan": {
      "visualAttention": {
        "score": 80,
        "description": ""
      },
      "auditoryAttention": {
        "score": 75,
        "description": ""
      }
    }
  },
  "executiveFunction": {
    "planning": 6,
    "impulseControl": 6,
    "description": ""
  },
  "learningPreferences": {
    "modalities": [],
    "description": ""
  },
  "learningChallengesSummary": {
    "attentionAndFocus": "",
    "memoryAndSpeed": "",
    "planningAndImpulsivity": "",
    "strengths": ""
  }
}


profile2= {
  "name": "Kid",
  "gender": "female",
  "age": 13,
  "diagnosis": "Attention Deficit Hyperactivity Disorder (ADHD)",
  "cognitiveDomainProfile": {
    "workingMemory": {
      "score": 85,
      "description": ""
    },
    "processingSpeed": {
      "score": 78,
      "description": ""
    },
    "attentionSpan": {
      "visualAttention": {
        "score": 80,
        "description": ""
      },
      "auditoryAttention": {
        "score": 75,
        "description": ""
      }
    }
  },
  "executiveFunction": {
    "planning": 6,
    "impulseControl": 6,
    "description": ""
  },
  "learningPreferences": {
    "modalities": [],
    "description": ""
  },
  "learningChallengesSummary": {
    "attentionAndFocus": "",
    "memoryAndSpeed": "",
    "planningAndImpulsivity": "",
    "strengths": ""
  }
}

profile3= {
  "name": "Kid",
  "gender": "female",
  "age": 13,
  "diagnosis": "Identified as Gifted and Talented (IQ > 130)",
  "cognitiveDomainProfile": {
    "workingMemory": {
      "score": 135,
      "description": ""
    },
    "processingSpeed": {
      "score": 140,
      "description": ""
    },
    "attentionSpan": {
      "visualAttention": {
        "score": 145,
        "description": ""
      },
      "auditoryAttention": {
        "score": 142,
        "description": ""
      }
    }
  },
  "executiveFunction": {
    "planning": 10,
    "impulseControl": 9,
    "description": ""
  },
  "learningPreferences": {
    "modalities": [],
    "description": ""
  },
  "learningChallengesSummary": {
    "attentionAndFocus": "",
    "memoryAndSpeed": "",
    "planningAndImpulsivity": "",
    "strengths": ""
  }
}


In [5]:
model = genai.GenerativeModel(
    model_name='models/gemini-1.5-pro',           
    system_instruction=system_instruction_prompt,  
    generation_config=generation_config,
       
)

pdf_paths = [
    "papers_profiles\Assessing executive functions in children.pdf",
    "papers_profiles\Assessments Related to the Physical, Affective and Cognitive Domains of Physical Literacy.pdf",
    "papers_profiles\Mapping Normative Trajectories of Cognitive.pdf",
    "papers_profiles\Domains of cognition and their assessment.pdf",
    "papers_profiles\Evidence-based Assessment of Cognitive Functioning.pdf",
    "papers_profiles\Measurement of Cognition for the.pdf",
    "papers_profiles\Auditory and Visual Attention in Normal and ADHD-Inattentive subtype Children.pdf"
]

# 5) Upload each PDF
pdf_files = [
    genai.upload_file(path=path, mime_type="application/pdf")
    for path in pdf_paths
]  # returns a list of File objects :contentReference[oaicite:0]{index=0}

# 6) Ask Gemini to read all three before answering
user_prompt = """
Please read the attached papers and use them to fill in the JSON skeleton below. {kid_profile}
The JSON object should be a complete and accurate representation of the child's cognitive profile including the modalities, based on the provided scores and the information in the papers. Ensure that all fields are populated according to the guidelines in the system instruction prompt.
"""

response = model.generate_content(
    pdf_files + [user_prompt.format(kid_profile=profile1)], 
)

print(response.text)

{"name": "Kid", "gender": "male", "age": 13, "diagnosis": "Not formally diagnosed, but exhibiting early symptoms suggestive of attention deficit hyperactivity disorder (ADHD)", "cognitiveDomainProfile": {"workingMemory": {"score": 85, "description": "Working memory, defined as the capacity to hold information in mind for adaptive use, appears to be within the average range for this child's age.  This includes the ability to maintain and manipulate both verbal and nonverbal information. While a score of 85 suggests adequate capacity for age, ongoing monitoring is recommended given the dynamic nature of working memory development during adolescence and its association with executive functions."}, "processingSpeed": {"score": 78, "description": "Processing speed, referring to the rate at which mental operations are performed, appears to be within the average range for this child's age.  While a score of 78 suggests adequate capacity for age, ongoing monitoring is recommended given the dyn

In [6]:
response = model.generate_content(
    pdf_files + [user_prompt.format(kid_profile=profile2)], 
)

print(response.text)

{"name": "Kid", "gender": "female", "age": 13, "diagnosis": "Attention Deficit Hyperactivity Disorder (ADHD)", "cognitiveDomainProfile": {"workingMemory": {"score": 85, "description": "Working memory, defined as the ability to hold information in consciousness for adaptive use across verbal, spatial, and other modalities, appears to be within the average range for this child's age.  Research suggests that working memory involves both maintenance and manipulation of information, and that it plays a crucial role in various cognitive functions, including executive function skills."}, "processingSpeed": {"score": 78, "description": "Processing speed, referring to the rate at which mental operations are performed, appears to be within the average range for this child.  Research suggests that slower processing speed can manifest as delayed reaction times and longer completion times for cognitive tasks.  It is considered a fundamental aspect of cognition that may influence performance on more

In [8]:
response = model.generate_content(
    pdf_files + [user_prompt.format(kid_profile=profile3)], 
)

print(response.text)

{"name": "Kid", "gender": "female", "age": 13, "diagnosis": "Identified as Gifted and Talented (IQ > 130)", "cognitiveDomainProfile": {"workingMemory": {"score": 135, "description": "Exceptional working memory abilities, significantly exceeding those of same-aged peers. Capable of holding and manipulating complex information in mind with ease, contributing to superior performance in tasks requiring active information processing."}, "processingSpeed": {"score": 140, "description": "Remarkably rapid processing speed, far surpassing the average for 13-year-olds. Demonstrates exceptional efficiency in performing mental operations, contributing to superior performance in timed tasks and overall cognitive assessments."}, "attentionSpan": {"visualAttention": {"score": 145, "description": "Exceptional visual attention skills, significantly exceeding those of same-aged peers. Demonstrates superior ability to focus on visual information and ignore distractions, contributing to enhanced performan

In [9]:
pdf_files

[genai.File({
     'name': 'files/cubnigh9oio6',
     'display_name': 'Assessing executive functions in children.pdf',
     'mime_type': 'application/pdf',
     'sha256_hash': 'MGRhZGQ3NTUwNGNjODdkNWIwZWVjY2ZlODllZjdjOTI3NjE3ZjFkZTYyZjJhMmZjODlmMjdkYzg4NDdmNWJlZg==',
     'size_bytes': '476328',
     'state': 'ACTIVE',
     'uri': 'https://generativelanguage.googleapis.com/v1beta/files/cubnigh9oio6',
     'create_time': '2025-04-21T18:34:29.060351Z',
     'expiration_time': '2025-04-23T18:34:29.024635236Z',
     'update_time': '2025-04-21T18:34:29.060351Z'}),
 genai.File({
     'name': 'files/a34kb8se79am',
     'display_name': 'Assessments Related to the Physical, Affective and Cognitive '
                     'Domains of Physical Literacy.pdf',
     'mime_type': 'application/pdf',
     'sha256_hash': 'ZDY3NzQyODdhMTZlMmY5MmMyZTljYTg0Njk2NzBjM2RmZDdkMjUwMWExNzI0MTQyODhmMTBmNDdjZWVhMmE3Zg==',
     'size_bytes': '1884494',
     'state': 'ACTIVE',
     'uri': 'https://generativelanguage.